## ⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the Bedrock boto3 setup notebook notebook. ⚠️ ⚠️ ⚠️

In [1]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"
%pip install --quiet "pillow>=9.5,<10"

  Using cached boto3-1.28.76-py3-none-any.whl.metadata (6.7 kB)
  Using cached awscli-1.29.76-py3-none-any.whl.metadata (11 kB)
  Using cached botocore-1.31.76-py3-none-any.whl.metadata (6.1 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.7.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Using cached PyYAML-6.0.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached pyasn1-0.5.0-py2.py3-none-any.whl (83 kB)
Using cached boto3-1.28.76-py3-none-any.whl (135 kB)
Using cached awscli-1.29.76-py3-none-any.whl (4.3 MB)
Using cached botocore-1.31.76-py3-none-any.whl (11.3 MB)
Using cached PyYAML-6.0.1-cp

## Create required clients

In [2]:
import json
import os
import sys
import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
# os.environ["AWS_PROFILE"] = ""
# os.environ["BEDROCK_ASSUME_ROLE"] = ""  # E.g. "arn:aws:..."

bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None))

model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Prepare prompt for LLM from different customer inputs

In [16]:
# If you'd like to try your own prompt, edit this parameter!
customer_input = "I am a male consultant in my 30s traveling to New York next week. What kind of outfit should I wear on my first day in the office? "

## Invoke the Anthropic Claude model

In [19]:
# first fetch possible styles and give options to customer
prompt = """

Human: list different style options for:
""" + customer_input + """

Assistant:"""
body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"
claudeResponse = ""

try:
    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    styles_response = response_body.get("completion")
    print(styles_response)

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

 Here are some different style options for a male consultant in his 30s traveling to New York for his first day in the office:

Business formal:
- A tailored two-piece suit in a neutral color like navy, gray, or black. Pair with a white or light blue dress shirt, leather dress shoes, and a conservative tie.

- A blazer or sport coat with dress pants, dress shirt, dress shoes, and tie. Opt for solids or subtle patterns/textures. 

Business casual:
- Dress pants or chinos with a button-up collar shirt or polo shirt. Pair with leather dress shoes or loafers. Can add a blazer or sweater on top.

- Dark jeans that are clean and pressed with a collared shirt, sweater or blazer, and leather shoes. Avoid distressed or ripped denim.

- A fitted sweater or cardigan over a collared shirt with trousers and leather shoes.

Smart casual:
- Dark jeans or chinos with a button-up shirt or casual collared shirt like an Oxford. Add loafers, Chelseas, or leather sneakers.

- A knit polo or fitted crewneck

## Preparing received response to generate images for each style look

In [20]:
# Prepare input for fetching images for each of style
styles = [s.strip() for s in (list(filter(None, styles_response.splitlines()))[1:-1])]
print(*styles, sep='\n')

Business formal:
- A tailored two-piece suit in a neutral color like navy, gray, or black. Pair with a white or light blue dress shirt, leather dress shoes, and a conservative tie.
- A blazer or sport coat with dress pants, dress shirt, dress shoes, and tie. Opt for solids or subtle patterns/textures.
Business casual:
- Dress pants or chinos with a button-up collar shirt or polo shirt. Pair with leather dress shoes or loafers. Can add a blazer or sweater on top.
- Dark jeans that are clean and pressed with a collared shirt, sweater or blazer, and leather shoes. Avoid distressed or ripped denim.
- A fitted sweater or cardigan over a collared shirt with trousers and leather shoes.
Smart casual:
- Dark jeans or chinos with a button-up shirt or casual collared shirt like an Oxford. Add loafers, Chelseas, or leather sneakers.
- A knit polo or fitted crewneck sweater with trousers or dark jeans and leather shoes.
- An unstructured blazer or sport coat over a T-shirt, collared shirt, or sweat

## Generating images for each style using stability.stable-diffusion-xl

In [ ]:
from PIL import Image
from IPython import display
from base64 import b64decode
import base64
import io
import json
import os
import sys

# Fetching images for each of style

os.makedirs("data", exist_ok=True)
images = []
for i, style in enumerate(styles):
    if len(style.split("-")) > 1:
        request = json.dumps({
            "text_prompts": (
                [{"text": "person wearing " + style.split("-")[1], "weight": 1.0}]
            ),
            "cfg_scale": 5,
            "seed": 5450,
            "steps": 70,
            "style_preset": "photographic",
        })
        modelId = "stability.stable-diffusion-xl"
        
        response = bedrock_runtime.invoke_model(body=request, modelId=modelId)
        response_body = json.loads(response.get("body").read())
        
        base_64_img_str = response_body["artifacts"][0].get("base64")
        display.display(display.Image(b64decode(base_64_img_str), width=200))